In [3]:
import os
from torch.utils.data import Dataset, DataLoader
import torch
import pandas
import numpy as np
from skimage import io, transform
class MyDataset(Dataset):
    def __init__(self, csv_file, directory, transform=None, use_cache=False, enable_cache=False, reduce_near_zero_samples=True): 
        self.df = pandas.read_csv(csv_file)
        self.timestamp = np.array(self.df.iloc[:,1])
        self.image_path = np.array(self.df.iloc[:,5])
        
        self.angle = np.array(self.df.iloc[:,6])
        
        if(reduce_near_zero_samples):
        
            to_delete_all = np.argwhere( (self.angle < 0.6) & (self.angle > -0.6 ))
            random_choise = np.random.choice(np.squeeze(to_delete_all), int(len(to_delete_all)/10), replace=False)
            self.image_path = np.delete(self.image_path,random_choise)
            self.angle = np.delete(self.angle,random_choise)


            to_delete_all = np.argwhere( (self.angle < 0.3) & (self.angle > -0.3 ))
            random_choise = np.random.choice(np.squeeze(to_delete_all), int(len(to_delete_all)/1.7), replace=False)
            self.image_path = np.delete(self.image_path,random_choise)
            self.angle = np.delete(self.angle,random_choise)


            to_delete_all = np.argwhere( (self.angle < 0.1) & (self.angle > -0.1 ))
            random_choise = np.random.choice(np.squeeze(to_delete_all), int(len(to_delete_all)/1.9), replace=False)
            self.image_path = np.delete(self.image_path,random_choise)
            self.angle = np.delete(self.angle,random_choise)

            to_delete_all = np.argwhere( (self.angle < 0.05) & (self.angle > -0.05 ))
            random_choise = np.random.choice(np.squeeze(to_delete_all), int(len(to_delete_all)/1.9), replace=False)
            self.image_path = np.delete(self.image_path,random_choise)
            self.angle = np.delete(self.angle,random_choise)


            to_delete_all = np.argwhere( (self.angle < 0.02) & (self.angle > -0.02 ))
            random_choise = np.random.choice(np.squeeze(to_delete_all), int(len(to_delete_all)/2.1), replace=False)
            self.image_path = np.delete(self.image_path,random_choise)
            self.angle = np.delete(self.angle,random_choise)
        
        
    
        self.tranform = transform
        self.dir = directory
        
        self.cached_data = []
        self.use_cache = use_cache
        self.enable_cache = enable_cache
    
    def __getitem__(self, index):
        if not self.use_cache:

            if torch.is_tensor(index):
                index = index.tolist()       
        
            y = self.angle[index]
            img_path =  os.path.join(self.dir,self.image_path[index])
            X = io.imread(img_path)
        
            if self.tranform:
                X = self.tranform(X)
        
            sample = {'X': X, 'y': y, 'Path':img_path}
            
            if(self.enable_cache):
                self.cached_data.append(sample)
            #print(len(self.cached_data))
            
            
        else:
            sample = self.cached_data[index]
            #print("cache used")

        return sample
    
    def get_custom_item(self, index):
       

        if torch.is_tensor(index):
            index = index.tolist()       
        
        y = self.angle[index]
        img_path =  os.path.join(self.dir,self.image_path[index])
        X = io.imread(img_path)
        
        if self.tranform:
            X = self.tranform(X)
        
        sample = {'X': X, 'y': y, 'Path':img_path}

        return sample
    
    def __len__(self):
        return len(self.angle)
    
    
    def set_use_cache(self, use_cache):
        if use_cache:
            print(len(self.cached_data))
            #self.cached_data = torch.stack(self.cached_data)
        #else:
            #self.cached_data = []
        self.use_cache = use_cache